In [0]:
pip install pyspark

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached pyspark-3.4.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('nestle').getOrCreate()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
schema={'DIM':StructType([StructField('LONG',StringType(),True),StructField('SHORT',StringType(),True),StructField('TYPE',StringType(),True)]),
'FACT':StructType([StructField('MKT_TAG',StringType(),True),StructField('PROD_TAG',StringType(),True),StructField('PER_TAG',StringType(),True),StructField('U',IntegerType(),True),StructField('V',IntegerType(),True),StructField('E',IntegerType(),True)]),
'FCT':StructType([StructField('TAG',StringType(),True),StructField('SHORT',StringType(),True),StructField('LONG',StringType(),True),StructField('DISPLAY_ORDER',IntegerType(),True),StructField('CURRENCY',StringType(),True),StructField('PRECISION',IntegerType(),True),StructField('DENOMINATOR',IntegerType(),True),StructField('MKT_AGGREGATION',StringType(),True),StructField('PROD_AGGREGATION',StringType(),True),StructField('PER_AGGREGATION',StringType(),True),StructField('FACT_TYPE',StringType(),True)]),
'MKT':StructType([StructField('TAG',StringType(),True),StructField('SHORT',StringType(),True),StructField('LONG',StringType(),True),StructField('DISPLAY_ORDER',IntegerType(),True),StructField('PARENT_TAG',StringType(),True),StructField('HIER_NUM',IntegerType(),True),StructField('HIER_NAME',StringType(),True),StructField('HIER_LEVEL_NUM',IntegerType(),True),StructField('HIER_LEVEL_NAME',StringType(),True)]),
'PER':StructType([StructField('TAG',StringType(),True),StructField('SHORT',StringType(),True),StructField('LONG',StringType(),True),StructField('DISPLAY_ORDER',IntegerType(),True),StructField('PARENT_TAG',StringType(),True),StructField('HIER_NUM',IntegerType(),True),StructField('HIER_NAME',StringType(),True),StructField('HIER_LEVEL_NUM',IntegerType(),True),StructField('HIER_LEVEL_NAME',StringType(),True),StructField('HALFYEAR',StringType(),True),StructField('HJYEAR',StringType(),True),StructField('MAT',StringType(),True),StructField('MATYEAR',StringType(),True),StructField('MONTH',StringType(),True),StructField('MONTHYEAR',StringType(),True),StructField('QUARTER',StringType(),True),StructField('QUARTERYEAR',StringType(),True),StructField('RELPER',StringType(),True),StructField('RELYEAR',StringType(),True),StructField('WEEKNR',StringType(),True),StructField('YEAR',StringType(),True),StructField('YTD',StringType(),True),StructField('YTDYEAR',StringType(),True)]),
'PROD':StructType([StructField('TAG',StringType(),True),StructField('SHORT',StringType(),True),StructField('LONG',StringType(),True),StructField('DISPLAY_ORDER',IntegerType(),True),StructField('PARENT_TAG',StringType(),True),StructField('HIER_NUM',IntegerType(),True),StructField('HIER_NAME',StringType(),True),StructField('HIER_LEVEL_NUM',IntegerType(),True),StructField('HIER_LEVEL_NAME',StringType(),True),StructField('SEG_MFR_BRD_SBRD_FORMAT_PACK_RANGE_PACK_SIZE_ITEM_level_0',StringType(),True),StructField('SEGMENT',StringType(),True),StructField('MANUFACTURER',StringType(),True),StructField('BRAND',StringType(),True),StructField('SUB_BRAND',StringType(),True),StructField('FORMAT',StringType(),True),StructField('PACKRANGE',StringType(),True),StructField('PACKSIZE',StringType(),True),StructField('MERGED_ITEM',StringType(),True),StructField('ITEM',StringType(),True),StructField('SUB_SEGMENT',StringType(),True),StructField('SUBSEGMENT_MFR_BRD_SBRD_level_0',StringType(),True),StructField('FORMAT_MFR_BRD_SBRD_PCKRANGE_level_0',StringType(),True),StructField('MFR_BRD_SBRD_level_0',StringType(),True),StructField('SEG_PCKRANGE_MFR_BRD_SBRD_level_0',StringType(),True),StructField('HEALTH_STAR_RATING',StringType(),True),StructField('HSR_MFR_BRD_SBRD_level_0',StringType(),True),StructField('CUISINE',StringType(),True),StructField('CUISINE_MFR_BRD_SBRD_level_0',StringType(),True),StructField('COLES_COMM',StringType(),True),StructField('COMPETITOR',StringType(),True),StructField('GTIN',StringType(),True),StructField('HBRAND_GENERIC',StringType(),True),StructField('TRAD_VS_CONTEMP',StringType(),True),StructField('WW_COMM',StringType(),True)])}

In [0]:
dbutils.widgets.text("input",",")
y=dbutils.widgets.get("input")
print(y)

,


In [0]:
dbutils.widgets.text("foldername",",")
z=dbutils.widgets.get("foldername")
ingestion_date=z[4:]+'-'+z[0:2]+'-'+z[2:4]
print(ingestion_date)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2853045885082523>:4
      2 z=dbutils.widgets.get("foldername")
      3 ingestion_date=z[4:]+'-'+z[0:2]+'_'+z[2:4]
----> 4 print(ingestiondate)

NameError: name 'ingestiondate' is not defined

In [0]:
y=y.replace('[','').replace(']','').replace('{','').replace('}','').replace('"type":"File"','').replace(',','').replace('"name":',' ').replace('"','').replace("'",'').split(' ')[1:]

In [0]:
options = {
"sfUrl": "https://qo30083.north-europe.azure.snowflakecomputing.com",
"sfUser": "dhanujapani",
"sfPassword": "@untP0lly",
"sfDatabase": "TESTNESTLE",
"sfSchema": "NESTLE",
"sfWarehouse": "COMPUTE_WH"
}

In [0]:


for i in y:
    if i[-3:]=='csv':
        file="dbfs:/mnt/standardisedfiles/"+i
        l=i.split('.')[0]
        df=spark.read.csv(file,schema=schema[l],header=True,sep='|')
        df=df.withColumn('filedate',lit(ingestion_date)).withColumn('ingestion_timestamp',lit(current_timestamp()))
        print(l)
        df.display()
        tablename=l
        df.write.format("SNOWFLAKE").options(**options).option("dbtable", tablename).mode('overwrite').options(header=True).save()



DIM


LONG,SHORT,TYPE,filedate,ingestion_timestamp
MARKET,MKT,DATA_KEY,2023-10-12,2023-05-11T14:22:46.469+0000
PRODUCT,PROD,DATA_KEY,2023-10-12,2023-05-11T14:22:46.469+0000
PERIOD,PER,DATA_KEY,2023-10-12,2023-05-11T14:22:46.469+0000
FACT,FCT,DATA_VALUE,2023-10-12,2023-05-11T14:22:46.469+0000


FACT


MKT_TAG,PROD_TAG,PER_TAG,U,V,E,filedate,ingestion_timestamp
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022029,22,170,2285,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022030,20,159,2156,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022031,19,150,2087,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022032,18,132,2033,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022033,18,131,1974,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022034,17,118,1941,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022035,17,132,1998,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022036,16,121,1956,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022037,16,129,1965,2023-10-12,2023-05-11T14:22:49.714+0000
M000000000000000511500000000000000020217,P000000000000032037100000000000000038034,W2022038,16,125,1842,2023-10-12,2023-05-11T14:22:49.714+0000


FCT


...

TAG,SHORT,LONG,DISPLAY_ORDER,PARENT_TAG,HIER_NUM,HIER_NAME,HIER_LEVEL_NUM,HIER_LEVEL_NAME,filedate,ingestion_timestamp
M000000000000000511500000000000000020217,Total Grocery,Total Grocery,1,null,1,5115AUSTRALIA GROCERY STATE,1,Total Grocery,2023-10-12,2023-05-11T14:22:54.737+0000
M000000000000000511500000000000000020518,NSW,Total Grocery NSW,2,M000000000000000511500000000000000020217,1,5115AUSTRALIA GROCERY STATE,2,State,2023-10-12,2023-05-11T14:22:54.737+0000
M000000000000000511500000000000000020521,QLD,Total Grocery QLD,3,M000000000000000511500000000000000020217,1,5115AUSTRALIA GROCERY STATE,2,State,2023-10-12,2023-05-11T14:22:54.737+0000
M000000000000000511500000000000000020519,SA/NT,Total Grocery SA/NT,4,M000000000000000511500000000000000020217,1,5115AUSTRALIA GROCERY STATE,2,State,2023-10-12,2023-05-11T14:22:54.737+0000
M000000000000000511500000000000000020522,TAS,Total Grocery TAS,5,M000000000000000511500000000000000020217,1,5115AUSTRALIA GROCERY STATE,2,State,2023-10-12,2023-05-11T14:22:54.737+0000
M000000000000000511500000000000000020523,VIC,Total Grocery VIC,6,M000000000000000511500000000000000020217,1,5115AUSTRALIA GROCERY STATE,2,State,2023-10-12,2023-05-11T14:22:54.737+0000
M000000000000000511500000000000000020520,WA,Total Grocery WA,7,M000000000000000511500000000000000020217,1,5115AUSTRALIA GROCERY STATE,2,State,2023-10-12,2023-05-11T14:22:54.737+0000


PER


TAG,SHORT,LONG,DISPLAY_ORDER,PARENT_TAG,HIER_NUM,HIER_NAME,HIER_LEVEL_NUM,HIER_LEVEL_NAME,HALFYEAR,HJYEAR,MAT,MATYEAR,MONTH,MONTHYEAR,QUARTER,QUARTERYEAR,RELPER,RELYEAR,WEEKNR,YEAR,YTD,YTDYEAR,filedate,ingestion_timestamp
W2022029,26/07/22,WE 26/07/22,148,null,null,null,null,null,null,null,MAT,MAT W41 2022,July,JUL 22,Q3,3. QU 22,P-2,null,29,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022030,02/08/22,WE 02/08/22,149,null,null,null,null,null,null,null,MAT,MAT W41 2022,July,JUL 22,Q3,3. QU 22,P-2,null,30,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022031,09/08/22,WE 09/08/22,150,null,null,null,null,null,null,null,MAT,MAT W41 2022,August,AUG 22,Q3,3. QU 22,P-1,null,31,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022032,16/08/22,WE 16/08/22,151,null,null,null,null,null,null,null,MAT,MAT W41 2022,August,AUG 22,Q3,3. QU 22,P-1,null,32,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022033,23/08/22,WE 23/08/22,152,null,null,null,null,null,null,null,MAT,MAT W41 2022,August,AUG 22,Q3,3. QU 22,P-1,null,33,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022034,30/08/22,WE 30/08/22,153,null,null,null,null,null,null,null,MAT,MAT W41 2022,August,AUG 22,Q3,3. QU 22,P-1,null,34,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022035,06/09/22,WE 06/09/22,154,null,null,null,null,null,null,null,MAT,MAT W41 2022,September,SEP 22,Q3,3. QU 22,P,null,35,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022036,13/09/22,WE 13/09/22,155,null,null,null,null,null,null,null,MAT,MAT W41 2022,September,SEP 22,Q3,3. QU 22,P,null,36,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022037,20/09/22,WE 20/09/22,156,null,null,null,null,null,null,null,MAT,MAT W41 2022,September,SEP 22,Q3,3. QU 22,P,null,37,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000
W2022038,27/09/22,WE 27/09/22,157,null,null,null,null,null,null,null,MAT,MAT W41 2022,September,SEP 22,Q3,3. QU 22,P,null,38,2022,YTD,YTD W41 2022,2023-10-12,2023-05-11T14:22:57.326+0000


PROD


TAG,SHORT,LONG,DISPLAY_ORDER,PARENT_TAG,HIER_NUM,HIER_NAME,HIER_LEVEL_NUM,HIER_LEVEL_NAME,SEG_MFR_BRD_SBRD_FORMAT_PACK_RANGE_PACK_SIZE_ITEM_level_0,SEGMENT,MANUFACTURER,BRAND,SUB_BRAND,FORMAT,PACKRANGE,PACKSIZE,MERGED_ITEM,ITEM,SUB_SEGMENT,SUBSEGMENT_MFR_BRD_SBRD_level_0,FORMAT_MFR_BRD_SBRD_PCKRANGE_level_0,MFR_BRD_SBRD_level_0,SEG_PCKRANGE_MFR_BRD_SBRD_level_0,HEALTH_STAR_RATING,HSR_MFR_BRD_SBRD_level_0,CUISINE,CUISINE_MFR_BRD_SBRD_level_0,COLES_COMM,COMPETITOR,GTIN,HBRAND_GENERIC,TRAD_VS_CONTEMP,WW_COMM,filedate,ingestion_timestamp
P000000000000032037100000000000000038034,RECIPE MIXES,RECIPE MIXES,1,null,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,1,SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM level 0,RECIPE MIXES,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000076306527,DRY,DRY,2,P000000000000032037100000000000000038034,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,2,SEGMENT,RECIPE MIXES,DRY,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000076306536,DRY MARS INC,DRY MARS INC,3,P000000000000032037200000000000076306527,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,3,MANUFACTURER,RECIPE MIXES,DRY,MARS INC,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000211567156,DRY MARS INC MASTERFOODS,DRY MARS INC MASTERFOODS,4,P000000000000032037200000000000076306536,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,4,BRAND,RECIPE MIXES,DRY,MARS INC,MASTERFOODS,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000211567786,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX,5,P000000000000032037200000000000211567156,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,5,SUB BRAND,RECIPE MIXES,DRY,MARS INC,MASTERFOODS,MASTERFOODS MASTER MIX,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000211566850,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX SACHET,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX SACHET,6,P000000000000032037200000000000211567786,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,6,FORMAT,RECIPE MIXES,DRY,MARS INC,MASTERFOODS,MASTERFOODS MASTER MIX,SACHET,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000211567218,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX SACHET 31-60G,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX SACHET 31-60G,7,P000000000000032037200000000000211566850,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,7,PACKRANGE,RECIPE MIXES,DRY,MARS INC,MASTERFOODS,MASTERFOODS MASTER MIX,SACHET,31-60G,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000211567720,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX SACHET 31-60G 40 G,DRY MARS INC MASTERFOODS MASTERFOODS MASTER MIX SACHET 31-60G 40 G,8,P000000000000032037200000000000211567218,1,32037SEG-MFR-BRD-SBRD-FORMAT-PACK RANGE-PACK SIZE-ITEM,8,PACKSIZE,RECIPE MIXES,DRY,MARS INC,MASTERFOODS,MASTERFOODS MASTER MIX,SACHET,31-60G,40 G,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023-10-12,2023-05-11T14:22:59.967+0000
P000000000000032037200000000000211665263,MASTERFOODS MASTER MIX TOMATO & GARLIC RISSOLE 40G,MASTERFOODS MASTER MIX TOMATO & GARLIC RISSOLE 